In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm

from utils import delete_nones, encode_ids

## Load and Preprocess data

In [ ]:
frame = pd.read_csv("../data/clear_data.scv")

In [ ]:
frame = delete_nones(frame)
frame = encode_ids(frame)

In [ ]:
# TODO: move into main
print('frame shape = ', frame.shape)
print('num unique drivers = ', np.array(frame.driver_id.unique()).shape[0])
print('num unique users = ', np.array(frame.user_id.unique()).shape[0])
print('num unique rides = ', np.array(frame.ride_id.unique()).shape[0])

## Compute Features

In [1]:
import xgboost as xgb
from sklearn import metrics
from sklearn.cross_validation import KFold
from sklearn.preprocessing import normalize
from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split


In [ ]:
scores = []

# read in data
dtrain = xgb.DMatrix(np.array(X_train_add[[]]), 
                     label=np.array(y_train))

dtest = xgb.DMatrix(np.array(X_test_add[[]]),
                    label=np.array(y_test))

# parameters
params = [{'booster': 'dart',
         'max_depth': 5, 
         'learning_rate': 0.1,
         'objective': 'multi:softmax', 
         'silent': True,
         'normalize_type': 'tree',
         'rate_drop': 0.1,
         'skip_drop': 0.5,
         'num_class': 3},
          
         {'booster': 'dart',
         'max_depth': 3, 
         'learning_rate': 0.01,
         'objective': 'multi:softmax', 
         'silent': True,
         'normalize_type': 'tree',
         'rate_drop': 0.1,
         'skip_drop': 0.8,
         'num_class': 3},
          
         {'booster': 'gblinear',
         'max_depth': 8, 
         'learning_rate': 0.001,
         'objective': 'multi:softmax', 
         'silent': True,
         'normalize_type': 'forest',
         'num_class': 3},
          
        {'booster': 'dart',
         'max_depth': 4, 
         'learning_rate': 0.1,
         'objective': 'multi:softmax', 
         'silent': True,
         'normalize_type': 'tree',
         'rate_drop': 0.2,
         'skip_drop': 0.6,
         'num_class': 3}
        ]

num_round = 50

for param_num, param in enumerate(params):
    print()
    print('##############################')
    print('###           {}           ###'.format(param_num))
    print('##############################')
    bst = xgb.train(param, dtrain)
    
    y_pred = bst.predict(dtest)
    scores.append(metrics.accuracy_score(y_test, y_pred))
    
    scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')